In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings as wr
from dataclasses import dataclass
from typing import Dict, Tuple
wr.filterwarnings('ignore')

In [20]:
df1 = pd.read_csv('Turbine_1.csv',low_memory=False)
df2 = pd.read_csv('Turbine_2.csv',low_memory=False)

In [21]:
@dataclass
class SSIConfig:
    fs: float              # sampling frequency [Hz]
    i: int                 # number of block rows (window size)
    n_modes: int           # model order (number of states to keep)
    # For SSI-DATA
    svd_weight: str = "plain"   # "plain" or "sqrt" on Hankel
    # For covariance estimation in SSI-COV
    max_lag: int = None    # if None, use 2*i



def _modal_from_A_C(A: np.ndarray, C: np.ndarray, fs: float) -> Dict:
    """
    Compute eigenvalues, continuous-time poles, frequencies and damping ratios,
    and mode shapes (output mode shapes) from discrete-time state matrices A, C.
    """
    eigvals, eigvecs = np.linalg.eig(A)  # eigvecs columns = right eigenvectors

    # Continuous-time poles: s = ln(lambda) / Ts  (Ts = 1/fs)
    Ts = 1.0 / fs
    s = np.log(eigvals) / Ts  # complex poles

    wn = np.abs(s)            # rad/s
    freqs_hz = wn / (2 * np.pi)
    zetas = -np.real(s) / wn
    # Fix any numerical issues
    zetas = np.real_if_close(zetas)
    freqs_hz = np.real_if_close(freqs_hz)

    # Mode shapes in output space: Phi = C * V
    Phi = C @ eigvecs   # shape (n_outputs, n_modes)

    # Normalize mode shapes (e.g. unit max magnitude per mode)
    for k in range(Phi.shape[1]):
        m = np.max(np.abs(Phi[:, k]))
        if m > 0:
            Phi[:, k] /= m

    return {
        "eigvals_discrete": eigvals,
        "poles_continuous": s,
        "freqs_hz": freqs_hz,
        "zetas": zetas,
        "mode_shapes": Phi,
    }



def ssi_data(Y: np.ndarray, cfg: SSIConfig) -> Dict:
    """
    Data-driven SSI (Hankel-based, MOESP-style).

    Parameters
    ----------
    Y   : (N, l) array, time series (zero-mean recommended; we also demean here)
          N = number of samples, l = number of channels
    cfg : SSIConfig

    Returns
    -------
    dict with keys:
        A, C                      – discrete-time state matrices
        eigvals_discrete          – eigenvalues of A
        poles_continuous          – continuous-time poles
        freqs_hz                  – natural frequencies [Hz]
        zetas                     – damping ratios
        mode_shapes               – output mode shapes (rows = sensors, cols = modes)
    """
    # --- basic setup ------------------------------------------------------
    Y = np.atleast_2d(Y)
    # demean each channel
    Y = Y - np.mean(Y, axis=0, keepdims=True)

    N, l = Y.shape
    i = cfg.i
    n = cfg.n_modes

    # number of columns in Hankel
    j = N - 2 * i + 1
    if j <= 0:
        raise ValueError("Not enough data for given i (block rows). Need N > 2*i - 1.")

    # --- build big Hankel matrix H with 2*i block rows -------------------
    # H has shape (2*i*l, j)
    # block row k (0-based) corresponds to y_{k+1} .. y_{k+j}
    H = np.zeros((2 * i * l, j), dtype=float)
    for k in range(2 * i):
        row_idx = slice(k * l, (k + 1) * l)
        # rows k .. k+j-1 in time, all channels
        blk = Y[k : k + j, :]          # shape (j, l)
        H[row_idx, :] = blk.T          # (l, j)

    # optional simple weighting on H before QR
    if cfg.svd_weight == "sqrt":
        H = H / np.sqrt(j)

    # --- QR factorisation of H^T  ----------------------------------------
    # H^T has shape (j, 2*i*l); economy QR gives:
    #   H^T = Q (j x 2*i*l) * R (2*i*l x 2*i*l)
    Q, R = np.linalg.qr(H.T, mode="reduced")
    R = R.T   # now R is (2*i*l, 2*i*l), upper triangular

    # partition R into past / future row blocks (each i*l rows)
    R_past   = R[0 : i * l, :]          # rows for "past"
    R_future = R[i * l : 2 * i * l, :]  # rows for "future"

    # projection-like matrix: take future rows, past columns
    # (same structure as the MATLAB Pi = R(i*l+1:2*i*l, 1:i*l))
    Pi = R_future[:, 0 : i * l]         # shape (i*l, i*l)

    # --- SVD of projection -> extended observability matrix --------------
    U, S, VT = np.linalg.svd(Pi, full_matrices=False)

    if n > U.shape[1]:
        raise ValueError(f"Requested model order n={n} exceeds SVD rank {U.shape[1]}.")

    U1 = U[:, :n]                # (i*l, n)
    S1 = S[:n]                   # (n,)

    # extended observability O_i (i*l x n)
    O_i = U1 @ np.diag(np.sqrt(S1))

    # --- extract C and estimate A via shift invariance --------------------
    # first l rows = C
    C = O_i[0:l, :]

    # shifted versions of O_i
    O_i0 = O_i[0 : (i - 1) * l, :]   # rows 0 .. (i-1)*l-1
    O_i1 = O_i[l : i * l, :]         # rows l .. i*l-1

    # least-squares: O_i0 * A ≈ O_i1
    A, _, _, _ = np.linalg.lstsq(O_i0, O_i1, rcond=None)

    # --- modal parameters from A, C --------------------------------------
    modal = _modal_from_A_C(A, C, cfg.fs)
    modal.update({"A": A, "C": C})
    return modal



def print_modal_summary(res: Dict, channel_names=None, label=""):
    """
    Print:
      Eigenvalues of A give:
        - natural frequencies
        - damping ratios
      Eigenvectors of A and rows of C give:
        - mode shapes
    """
    eigvals = res["eigvals_discrete"]
    freqs  = res["freqs_hz"]
    zetas  = res["zetas"]
    Phi    = res["mode_shapes"]   # (n_outputs, n_modes)
    A      = res["A"]
    C      = res["C"]

    if label:
        print(f"\n===== {label} =====")

    print("\nEigenvalues of A give:")
    print("  Natural frequencies and damping ratios per mode:\n")
    for k, lam in enumerate(eigvals):
        f_k = np.real(freqs[k])
        z_k = np.real(zetas[k])
        print(f"  Mode {k+1}:")
        print(f"    λ_{k+1} = {lam.real:.4e} + {lam.imag:.4e}j")
        print(f"    f_{k+1} = {f_k:.4f} Hz")
        print(f"    ζ_{k+1} = {z_k:.4f}\n")

    print("Eigenvectors of A and rows of C give:")
    print("  Mode shapes in output (sensor) space (rows = sensors, cols = modes).")
    if channel_names is not None and len(channel_names) == Phi.shape[0]:
        print("\n  Mode shapes (real part):")
        header = "Sensor".ljust(20) + "".join([f"Mode {k+1:>10}" for k in range(Phi.shape[1])])
        print(header)
        for i, name in enumerate(channel_names):
            row_vals = "".join([f"{np.real(Phi[i, k]):10.3f}" for k in range(Phi.shape[1])])
            print(name.ljust(20) + row_vals)
    else:
        print("  Phi shape:", Phi.shape, " (use res['mode_shapes'])")


In [22]:
fs = 62.5  # Hz

channels = ['B1_root_edge', 'B1_root_span', 'B1_root_flap', 'B1_tip_edge', 'B1_tip_span', 'B1_tip_flap']

Y = df1[channels].to_numpy(dtype=float)

cfg = SSIConfig(fs=fs, i=1250, n_modes=8)

res_data = ssi_data(Y, cfg)


print_modal_summary(res_data, channel_names=channels, label="SSI-DATA")



===== SSI-DATA =====

Eigenvalues of A give:
  Natural frequencies and damping ratios per mode:

  Mode 1:
    λ_1 = 9.9973e-01 + 2.2382e-02j
    f_1 = 0.2227 Hz
    ζ_1 = 0.0007

  Mode 2:
    λ_2 = 9.9973e-01 + -2.2382e-02j
    f_2 = 0.2227 Hz
    ζ_2 = 0.0007

  Mode 3:
    λ_3 = 9.9970e-01 + 2.0826e-02j
    f_3 = 0.2072 Hz
    ζ_3 = 0.0039

  Mode 4:
    λ_4 = 9.9970e-01 + -2.0826e-02j
    f_4 = 0.2072 Hz
    ζ_4 = 0.0039

  Mode 5:
    λ_5 = 9.9968e-01 + 2.1579e-02j
    f_5 = 0.2147 Hz
    ζ_5 = 0.0039

  Mode 6:
    λ_6 = 9.9968e-01 + -2.1579e-02j
    f_6 = 0.2147 Hz
    ζ_6 = 0.0039

  Mode 7:
    λ_7 = 9.9993e-01 + 4.0838e-04j
    f_7 = 0.0041 Hz
    ζ_7 = 0.1677

  Mode 8:
    λ_8 = 9.9993e-01 + -4.0838e-04j
    f_8 = 0.0041 Hz
    ζ_8 = 0.1677

Eigenvectors of A and rows of C give:
  Mode shapes in output (sensor) space (rows = sensors, cols = modes).

  Mode shapes (real part):
Sensor              Mode          1Mode          2Mode          3Mode          4Mode          5Mo

In [23]:
fs = 62.5  # Hz

channels = ['B1_root_edge', 'B1_root_span', 'B1_root_flap', 'B1_tip_edge', 'B1_tip_span', 'B1_tip_flap']

Y = df2[channels].to_numpy(dtype=float)

cfg = SSIConfig(fs=fs, i=1250, n_modes=8)

res_data = ssi_data(Y, cfg)


print_modal_summary(res_data, channel_names=channels, label="SSI-DATA")



===== SSI-DATA =====

Eigenvalues of A give:
  Natural frequencies and damping ratios per mode:

  Mode 1:
    λ_1 = 9.9975e-01 + 2.0703e-02j
    f_1 = 0.2060 Hz
    ζ_1 = 0.0016

  Mode 2:
    λ_2 = 9.9975e-01 + -2.0703e-02j
    f_2 = 0.2060 Hz
    ζ_2 = 0.0016

  Mode 3:
    λ_3 = 9.9937e-01 + 2.1293e-02j
    f_3 = 0.2119 Hz
    ζ_3 = 0.0190

  Mode 4:
    λ_4 = 9.9937e-01 + -2.1293e-02j
    f_4 = 0.2119 Hz
    ζ_4 = 0.0190

  Mode 5:
    λ_5 = 9.9911e-01 + 1.8103e-02j
    f_5 = 0.1804 Hz
    ζ_5 = 0.0399

  Mode 6:
    λ_6 = 9.9911e-01 + -1.8103e-02j
    f_6 = 0.1804 Hz
    ζ_6 = 0.0399

  Mode 7:
    λ_7 = 9.9977e-01 + 7.9373e-04j
    f_7 = 0.0082 Hz
    ζ_7 = 0.2733

  Mode 8:
    λ_8 = 9.9977e-01 + -7.9373e-04j
    f_8 = 0.0082 Hz
    ζ_8 = 0.2733

Eigenvectors of A and rows of C give:
  Mode shapes in output (sensor) space (rows = sensors, cols = modes).

  Mode shapes (real part):
Sensor              Mode          1Mode          2Mode          3Mode          4Mode          5Mo

In [41]:
df =  pd.read_excel('mode_shapes.xlsx',sheet_name='Mode_Shapes_B1')

In [42]:
df_T1_modes = df.iloc[0:6, 1:9]   # sensors = rows, Modes 1–8 = columns
phi_T1 = df_T1_modes.to_numpy(dtype=float)
# rows 11 to 16 contain mode shapes for Turbine 2
df_T2_modes = df.iloc[11:16+1, 1:9]
phi_T2 = df_T2_modes.to_numpy(dtype=float)

In [44]:
def MAC(phi_a, phi_b):
    num = np.abs(np.dot(phi_a.T, phi_b))**2
    den = np.dot(phi_a.T, phi_a) * np.dot(phi_b.T, phi_b)
    return num / den

mac_mode1 = MAC(phi_T1[:,0], phi_T2[:,0])
mac_mode2 = MAC(phi_T1[:,1], phi_T2[:,1])
mac_mode3 = MAC(phi_T1[:,2], phi_T2[:,2])
print("MAC Mode 1:", mac_mode1)
print("MAC Mode 2:", mac_mode2)
print("MAC Mode 3:", mac_mode3)

MAC Mode 1: 0.8954445927269077
MAC Mode 2: 0.8954445927269077
MAC Mode 3: 0.9206213717850849
